In [125]:
import requests
import random
import json
from tqdm import tqdm

In [126]:
with open("generated_queries.txt", "r") as queries_file:
    queries = eval(queries_file.read())

In [127]:
random.shuffle(queries)

## Altera Função

In [128]:
# curl -X POST "127.0.0.1:9200/*/_close"; curl -XPUT -H "Content-Type: application/json" -d '{
#   "index": {
#     "similarity": {
#       "default": {
#         "type": "BM25"
#       }
#     }
#   }
# }' http://localhost:9200/*/_settings; curl -X POST "127.0.0.1:9200/*/_open"

In [3]:
# curl -X POST "127.0.0.1:9200/*/_close"; curl -XPUT -H "Content-Type: application/json" -d '{
#   "index": {
#     "similarity": {
#       "default": {
#         "type": "DFR",
#         "basic_model": "g",
#         "after_effect": "b",
#         "normalization": "no"
#       }
#     }
#   }
# }' http://localhost:9200/*/_settings; curl -X POST "127.0.0.1:9200/*/_open"

In [ ]:
curl -X POST "127.0.0.1:9200/*/_close"; curl -XPUT -H "Content-Type: application/json" -d '{
  "index": {
    "similarity": {
      "default": {
        "type": "LMDirichlet"
      }
    }
  }
}' http://localhost:9200/*/_settings; curl -X POST "127.0.0.1:9200/*/_open"

## Submete consultas

In [153]:
strategy="dirisparse" #bm25sparse dfrsparse dense

In [154]:
queries

[{'query': 'ZULMIRA ALVES PINTO ASSALVE',
  'origin': 'aduna',
  'id': '6e7e2656-9b98-47b4-8c17-7357223d6750'},
 {'query': 'advocacia administrativa não configurada',
  'origin': 'auto_complete',
  'id': 'e6ca073b-5bf6-4d05-933f-738f95215f8e'},
 {'query': 'defesa previa',
  'origin': 'auto_complete',
  'id': 'c3688899-a934-4a1c-b9cc-b2a7e7d1b959'},
 {'query': 'títulos da dívida pública',
  'origin': 'auto_complete',
  'id': '98eec51a-3010-4542-b4fe-d936e8c4bffc'},
 {'query': 'tipos de contrato',
  'origin': 'auto_complete',
  'id': '6e9d27ce-344b-4277-8de5-8022b7232b5d'},
 {'query': 'PROJETO LOTUS INFANTIL LTDA',
  'origin': 'aduna',
  'id': 'f68ac71b-4b95-49a7-9c4c-62585ae108da'},
 {'query': 'nunca sequer havia',
  'origin': 'auto_complete',
  'id': 'b90c2f29-488a-4148-8722-bd212bad1985'},
 {'query': 'juros superiores a',
  'origin': 'auto_complete',
  'id': 'e94487c0-0d4c-4658-aa56-364254051e2d'},
 {'query': 'microrregião ervália',
  'origin': 'lins',
  'id': '7c1dd922-ceb0-4004-8059

In [155]:
feedback_data = []
for q in tqdm(queries):
    try:
        query = q["query"].replace("/", "")
        params = {'query': query,  'page': 1, 'sid': '123'}
        service_response = requests.get('http://127.0.0.1:9000/services/search', params)
        results = json.loads(service_response.text)
        feedback_line = {}
        feedback_line["text"] = query
        feedback_line["_id"] = q["id"]
        feedback_line["corresponding"] = []
        # seleciona os 2 primeiros de cada estratégia
        results = sorted(json.loads(service_response.text)["documents"], key=lambda x: x["rank_number"])
        if not len(results):
            print("e", len(results))
        for row in results:
            feedback_line["corresponding"].append({"text": row["conteudo"].encode('utf-8').decode("utf-8"), "_id": row["id"]})
        feedback_data.append(feedback_line)
    except Exception as err:
        print(err)
        print(service_response.text)
        continue

100%|███████████████████████████████████████████████████| 50/50 [01:31<00:00,  1.82s/it]


In [156]:
with open(f"feedback_queries_{strategy}.json", "w") as output:
    output.write(json.dumps(feedback_data, ensure_ascii=False))

## Merge rankings

In [157]:
with open(f"feedback_queries_dense.json", "r") as input_file:
    feedback_data_dense = eval(input_file.read())
    
with open(f"feedback_queries_dirisparse.json", "r") as input_file:
    feedback_data_dirisparse = eval(input_file.read())
    
with open(f"feedback_queries_bm25sparse.json", "r") as input_file:
    feedback_data_bm25sparse = eval(input_file.read())

In [158]:
feedback_data_dense_dict = {}
for line in feedback_data_dense:
    feedback_data_dense_dict[line["text"]] = line
    
feedback_data_dirisparse_dict = {}
for line in feedback_data_dirisparse:
    feedback_data_dirisparse_dict[line["text"]] = line
    
feedback_data_bm25sparse_dict = {}
for line in feedback_data_bm25sparse:
    feedback_data_bm25sparse_dict[line["text"]] = line

In [159]:
def format_text(text):
    import re
    text = re.sub('\n+', '\n', text)
    return text.replace("\n", " ")

In [160]:
feedback_data_final = []
for q in tqdm(queries):
    try:
        query = q["query"].replace("/", "")

        feedback_line = {}
        feedback_line["text"] = "CONSULTA: " + query
        feedback_line["query"] = query
        feedback_line["_id"] = q["id"]
        feedback_line["corresponding"] = []
        counter = 0
        included = []
        max_candidate = 0
        
        for origin, feedback_list in zip(["dense", "dirisparse", "bm25sparse"], [feedback_data_dense_dict, feedback_data_dirisparse_dict, feedback_data_bm25sparse_dict]):
            max_candidate+= len(feedback_list[query]["corresponding"])
        if max_candidate < 6:
            continue

        #combina os corresponding de cada estratégia  
        while len(feedback_line["corresponding"]) < 6:
            counter = 0
            for origin, feedback_list in zip(["dense", "dirisparse", "bm25sparse"], [feedback_data_dense_dict, feedback_data_dirisparse_dict, feedback_data_bm25sparse_dict]):
                for entry in feedback_list[query]["corresponding"]:
                    # incluir a posição do elemento naquele ranking de origem 
                    if query + "--" + entry["_id"] in included:
                        continue
                    if counter == 2:
                        counter = 0
                        break
                    entry["text"] = format_text(entry["text"])
                    entry["origin"] = origin
                    feedback_line["corresponding"].append(entry) 
                    counter+= 1
                    included.append(query + "--" + entry["_id"])
        feedback_data_final.append(feedback_line)
    except Exception as err:
        print(err)
        raise err
        continue

100%|█████████████████████████████████████████████████| 50/50 [00:00<00:00, 1098.99it/s]


In [178]:
len(feedback_data_final)

50

In [179]:
with open(f"feedback_queries_final.json", "w") as output:
    output.write(json.dumps(feedback_data_final, ensure_ascii=False))

In [180]:
feedback_data_final

[{'text': 'CONSULTA: ZULMIRA ALVES PINTO ASSALVE',
  'query': 'ZULMIRA ALVES PINTO ASSALVE',
  '_id': '6e7e2656-9b98-47b4-8c17-7357223d6750',
  'corresponding': [{'text': ' Documento:20000020845 Poder Judiciário TRIBUNAL REGIONAL FEDERAL DA 2ª REGIÃO Agravo de Instrumento  Nº 5003670-02.2019.4.02.0000/RJ AGRAVANTE: ZULMIRA VARGAS FRANCO NETTOAGRAVANTE: ZILPA GUEDES DE SOUZAAGRAVANTE: ZELY PEREIRA DIASAGRAVANTE: ZULEIKA ALVES DA CRUZAGRAVANTE: ZILDA MARTINS BARBOSAAGRAVADO: UNIVERSIDADE FEDERAL DO RIO DE JANEIRO - UFRJAGRAVADO: PRÓ REITOR DE GESTÃO DE PESSOAS - UNIVERSIDADE FEDERAL DO RIO DE JANEIRO - UFRJ - Rio de JaneiroAGRAVADO: Reitor - UNIVERSIDADE FEDERAL DO RIO DE JANEIRO - UFRJ - Rio de Janeiro DESPACHO/DECISÃO Cuida-se de Agravo de Instrumento interposto por ZELY PEREIRA DIAS, ZILDA MARTINS BARBOSA, ZILPA GUEDES DE SOUZA, ZULEIKA ALVES DA CRUZ e ZULMIRA VARGAS FRANCO NETTO\xa0em face da\xa0decisão que indeferiu pedido de gratuidade de justiça pleiteado\xa0(evento 3 dos autos de

# Distribuição 

In [181]:
feedback_elves = []
feedback_rodrygo = []
feedback_rafael = []
feedback_rennan = []

In [182]:
for idx, i in enumerate(feedback_data_final):
    if idx % 2 == 0:
        feedback_elves.append(i)
    if idx % 2 == 1:
        feedback_rodrygo.append(i)
    if idx < len(feedback_data_final) /2 :
        feedback_rafael.append(i)
    if idx >= len(feedback_data_final) /2 :
        feedback_rennan.append(i)
    

In [183]:
for person, data_list in zip(["elves", "rodrygo", "rafael", "rennan"], [feedback_elves, feedback_rodrygo, feedback_rafael, feedback_rennan]):
    with open(f"feedback_{person}.json", "w") as output:
        output.write(json.dumps(data_list, ensure_ascii=False))

#### To Do
